In [1]:
import copy
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, r'D:\Учеба\Практика\Диплом\gamma\gamma_4\Scripts')
sys.path.insert(0, r'/home/danis/work/python_works/gamma_4/Scripts')
from cross_sections_processing import dataframe_to_list, edge_to_dict, interpolate_cs_on_general_grid, to_log

In [55]:
WINDOWS_CS_PATH = r'D:\Учеба\Практика\Диплом\gamma\gamma_4\Scripts'

In [56]:
# Считывание данных
cross_sections_df = pd.read_excel(r'D:\Учеба\Практика\Диплом\gamma\gamma_4\Scripts\DATA\cross_sections.xlsx', 
                                  header=None, sheet_name=['TOTAL', 'ABSORBT'], dtype=np.float64)

total = cross_sections_df['TOTAL']
absorbt = cross_sections_df['ABSORBT']

In [57]:
CS_total = dataframe_to_list(total, 'c')
CS_absorbt = dataframe_to_list(absorbt, 'c')
CS_init_mesh = dataframe_to_list(total, 'e')

In [58]:
# Список со списками энергий каждого элемента
energies = dataframe_to_list(total, 'e')

min_en = 1e-3

# Создание списка с энергиями всех скачков
edge_energies = []

for list_ in energies:
    for energy in list_:
        if energy not in edge_energies:
            if energy >= min_en:
                if list_.count(energy) == 2:
                    edge_energies.append(energy)
                    
edge_energies.sort()

In [59]:
edge_energies

[0.0010065,
 0.0010073,
 0.0010214,
 0.001024,
 0.001026,
 0.0010305,
 0.0010346,
 0.0010486,
 0.0010505,
 0.0010554,
 0.001057,
 0.0010595,
 0.0010596,
 0.0010618,
 0.001064,
 0.0010708,
 0.0010727,
 0.0010858,
 0.0010861,
 0.0011048,
 0.001106,
 0.0011129,
 0.001122,
 0.0011227,
 0.0011303,
 0.0011322,
 0.0011378,
 0.001147,
 0.0011504,
 0.0011623,
 0.0011672,
 0.0011705,
 0.0011844,
 0.0011897,
 0.0011944,
 0.0011964,
 0.0011998,
 0.0012048,
 0.0012101,
 0.0012228,
 0.0012284,
 0.0012316,
 0.0012366,
 0.0012446,
 0.0012495,
 0.0012554,
 0.0012599,
 0.0012642,
 0.0012729,
 0.0012734,
 0.001279,
 0.0012871,
 0.0012907,
 0.0012945,
 0.0012995,
 0.0013106,
 0.0013159,
 0.0013195,
 0.0013257,
 0.0013283,
 0.0013366,
 0.0013469,
 0.0013504,
 0.0013553,
 0.0013596,
 0.0013621,
 0.0013657,
 0.0013705,
 0.0013928,
 0.0013952,
 0.0014023,
 0.0014077,
 0.0014099,
 0.001412,
 0.0014199,
 0.0014282,
 0.0014386,
 0.001455,
 0.0014616,
 0.0014696,
 0.0014698,
 0.0014789,
 0.0014801,
 0.0014812,
 0

In [60]:
edge_energies_with_first_point = copy.copy(edge_energies)
edge_energies_with_first_point.insert(0, min_en)

In [61]:
edge_energies_with_first_point

[0.001,
 0.0010065,
 0.0010073,
 0.0010214,
 0.001024,
 0.001026,
 0.0010305,
 0.0010346,
 0.0010486,
 0.0010505,
 0.0010554,
 0.001057,
 0.0010595,
 0.0010596,
 0.0010618,
 0.001064,
 0.0010708,
 0.0010727,
 0.0010858,
 0.0010861,
 0.0011048,
 0.001106,
 0.0011129,
 0.001122,
 0.0011227,
 0.0011303,
 0.0011322,
 0.0011378,
 0.001147,
 0.0011504,
 0.0011623,
 0.0011672,
 0.0011705,
 0.0011844,
 0.0011897,
 0.0011944,
 0.0011964,
 0.0011998,
 0.0012048,
 0.0012101,
 0.0012228,
 0.0012284,
 0.0012316,
 0.0012366,
 0.0012446,
 0.0012495,
 0.0012554,
 0.0012599,
 0.0012642,
 0.0012729,
 0.0012734,
 0.001279,
 0.0012871,
 0.0012907,
 0.0012945,
 0.0012995,
 0.0013106,
 0.0013159,
 0.0013195,
 0.0013257,
 0.0013283,
 0.0013366,
 0.0013469,
 0.0013504,
 0.0013553,
 0.0013596,
 0.0013621,
 0.0013657,
 0.0013705,
 0.0013928,
 0.0013952,
 0.0014023,
 0.0014077,
 0.0014099,
 0.001412,
 0.0014199,
 0.0014282,
 0.0014386,
 0.001455,
 0.0014616,
 0.0014696,
 0.0014698,
 0.0014789,
 0.0014801,
 0.001

In [62]:
# Создание основной сетки энергий
base_enegries = [round(((edge_energies_with_first_point[i] + edge_energies_with_first_point[i+1]) / 2), 10) for i in range(len(edge_energies_with_first_point)-1)]

In [63]:
base_enegries

[0.00100325,
 0.0010069,
 0.00101435,
 0.0010227,
 0.001025,
 0.00102825,
 0.00103255,
 0.0010416,
 0.00104955,
 0.00105295,
 0.0010562,
 0.00105825,
 0.00105955,
 0.0010607,
 0.0010629,
 0.0010674,
 0.00107175,
 0.00107925,
 0.00108595,
 0.00109545,
 0.0011054,
 0.00110945,
 0.00111745,
 0.00112235,
 0.0011265,
 0.00113125,
 0.001135,
 0.0011424,
 0.0011487,
 0.00115635,
 0.00116475,
 0.00116885,
 0.00117745,
 0.00118705,
 0.00119205,
 0.0011954,
 0.0011981,
 0.0012023,
 0.00120745,
 0.00121645,
 0.0012256,
 0.00123,
 0.0012341,
 0.0012406,
 0.00124705,
 0.00125245,
 0.00125765,
 0.00126205,
 0.00126855,
 0.00127315,
 0.0012762,
 0.00128305,
 0.0012889,
 0.0012926,
 0.001297,
 0.00130505,
 0.00131325,
 0.0013177,
 0.0013226,
 0.001327,
 0.00133245,
 0.00134175,
 0.00134865,
 0.00135285,
 0.00135745,
 0.00136085,
 0.0013639,
 0.0013681,
 0.00138165,
 0.001394,
 0.00139875,
 0.001405,
 0.0014088,
 0.00141095,
 0.00141595,
 0.00142405,
 0.0014334,
 0.0014468,
 0.0014583,
 0.0014656,
 0.0

In [64]:
tail_energies = np.linspace(edge_energies[-1], 2, 309)

In [65]:
tail_energies

array([0.14268   , 0.14871026, 0.15474052, 0.16077078, 0.16680104,
       0.1728313 , 0.17886156, 0.18489182, 0.19092208, 0.19695234,
       0.2029826 , 0.20901286, 0.21504312, 0.22107338, 0.22710364,
       0.2331339 , 0.23916416, 0.24519442, 0.25122468, 0.25725494,
       0.26328519, 0.26931545, 0.27534571, 0.28137597, 0.28740623,
       0.29343649, 0.29946675, 0.30549701, 0.31152727, 0.31755753,
       0.32358779, 0.32961805, 0.33564831, 0.34167857, 0.34770883,
       0.35373909, 0.35976935, 0.36579961, 0.37182987, 0.37786013,
       0.38389039, 0.38992065, 0.39595091, 0.40198117, 0.40801143,
       0.41404169, 0.42007195, 0.42610221, 0.43213247, 0.43816273,
       0.44419299, 0.45022325, 0.45625351, 0.46228377, 0.46831403,
       0.47434429, 0.48037455, 0.48640481, 0.49243506, 0.49846532,
       0.50449558, 0.51052584, 0.5165561 , 0.52258636, 0.52861662,
       0.53464688, 0.54067714, 0.5467074 , 0.55273766, 0.55876792,
       0.56479818, 0.57082844, 0.5768587 , 0.58288896, 0.58891

In [66]:
# Объединение основных энергий и энергий К скачков
general_grid = []
general_grid.extend(edge_energies)
general_grid.extend(edge_energies)
general_grid.append(min_en)

for val in base_enegries:
    if val not in general_grid:
        general_grid.append(val)
        
for val in tail_energies:
    if val not in general_grid:
        general_grid.append(val)
        
general_grid.sort()

In [67]:
len(general_grid)

2001

In [69]:
for i in general_grid:
    print(i)

0.001
0.00100325
0.0010065
0.0010065
0.0010069
0.0010073
0.0010073
0.00101435
0.0010214
0.0010214
0.0010227
0.001024
0.001024
0.001025
0.001026
0.001026
0.00102825
0.0010305
0.0010305
0.00103255
0.0010346
0.0010346
0.0010416
0.0010486
0.0010486
0.00104955
0.0010505
0.0010505
0.00105295
0.0010554
0.0010554
0.0010562
0.001057
0.001057
0.00105825
0.0010595
0.0010595
0.00105955
0.0010596
0.0010596
0.0010607
0.0010618
0.0010618
0.0010629
0.001064
0.001064
0.0010674
0.0010708
0.0010708
0.00107175
0.0010727
0.0010727
0.00107925
0.0010858
0.0010858
0.00108595
0.0010861
0.0010861
0.00109545
0.0011048
0.0011048
0.0011054
0.001106
0.001106
0.00110945
0.0011129
0.0011129
0.00111745
0.001122
0.001122
0.00112235
0.0011227
0.0011227
0.0011265
0.0011303
0.0011303
0.00113125
0.0011322
0.0011322
0.001135
0.0011378
0.0011378
0.0011424
0.001147
0.001147
0.0011487
0.0011504
0.0011504
0.00115635
0.0011623
0.0011623
0.00116475
0.0011672
0.0011672
0.00116885
0.0011705
0.0011705
0.00117745
0.0011844
0.0011844


In [70]:
def interpolate_with_edge(dataframe, general_grid):
    materials = dataframe_to_list(dataframe)
    gg = general_grid
    ics = interpolate_cs_on_general_grid(dataframe, gg)
    
    ggl = [math.log(val) for val in gg]
    dfl = to_log(dataframe)
    icsl = interpolate_cs_on_general_grid(dfl, ggl)
    
    for list_ in icsl:
        for p, val in enumerate(list_):
            list_[p] = math.exp(val)
            
    for i in range(len(materials)):
        print(i)
        for j in range(len(edge_to_dict(i, 'energy', dataframe))):
            if edge_to_dict(i, 'energy', dataframe)[j] >= min_en:
                icsl[i][gg.index(edge_to_dict(i, 'energy', dataframe)[j])] = edge_to_dict(i, 'left', dataframe)[j]
                icsl[i][gg.index(edge_to_dict(i, 'energy', dataframe)[j]) + 1] = edge_to_dict(i, 'right', dataframe)[j]
                
    return icsl

In [71]:
cs_total = interpolate_with_edge(total, general_grid)
cs_absorbt = interpolate_with_edge(absorbt, general_grid)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [72]:
cs_total_npy = np.array(cs_total)
cs_absorbt_npy = np.array(cs_absorbt)
genaral_grid_npy = np.array(general_grid)

In [73]:
CROSS_SECTION_PROCESSED = np.array([general_grid, cs_total_npy, cs_absorbt_npy], dtype=object)

In [74]:
CROSS_SECTION_INIT = np.array([CS_init_mesh, CS_total, CS_absorbt], dtype=object)

In [75]:
np.save('CROSS_SECTION_PROCESSED_mesh_02.npy', CROSS_SECTION_PROCESSED)

In [44]:
np.save('CROSS_SECTION_INIT.npy', CROSS_SECTION_INIT)